In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 준비
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma DB 준비
vectorstore = Chroma(
    collection_name="policy_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_db_policy"
)


In [3]:
import os
import json
from openai import OpenAI


In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
# 1단계 : 사전 차단
def is_policy_related_query(query: str) -> bool:
    """
    사용자 질문이 정책 추천 도메인과 관련이 있는지 규칙 기반으로 판단합니다.
    (1단계: Guardrail)
    """
    # 긍정 키워드: 이 단어들이 포함되면 관련 질문으로 간주
    policy_keywords = [
        '정책', '지원', '혜택', '대출', '주거', '일자리', '금융', '교육',
        '청년', '신청', '자격', '소득', '보증금', '월세', '취업', '창업'
    ]
    
    # 부정 키워드: 이 단어들이 포함되면 관련 없는 질문으로 간주
    unrelated_keywords = [
        '날씨', '맛집', '노래', '영화', '드라마', '안녕', '반가워', 'ㅎㅇ',
        '사랑', '여행', '게임', '스포츠'
    ]

    query_lower = query.lower()

    # 부정 키워드가 하나라도 있으면 즉시 False 반환
    if any(keyword in query_lower for keyword in unrelated_keywords):
        return False

    # 긍정 키워드가 하나라도 있으면 True 반환
    if any(keyword in query_lower for keyword in policy_keywords):
        return True
        
    # 위 두 조건에 모두 해당하지 않으면, 관련 없는 것으로 간주 (보수적 접근)
    return False

# 2단계 : 상세 정보 추출
def extract_structured_data_with_llm(query: str) -> dict:
    from datetime import datetime
    current_date_int = int(datetime.now().strftime('%Y%m%d'))
    
    system_prompt = f"""
    당신은 대한민국 청년 정책에 대한 사용자 질문을 분석하여, Chroma 데이터베이스 검색 필터로 사용할 JSON을 생성하는 AI 어시스턴트입니다.
    
    # 기본 원칙:
    - 사용자가 기간을 명시하지 않으면, 반드시 '현재 신청 가능한' 정책을 찾는 것을 기본값으로 삼아야 합니다.
    - 이를 위해, 현재 날짜({current_date_int})를 기준으로 `start_date_int`와 `end_date_int`를 비교하는 필터를 생성해야 합니다.
    - **중요**: Chroma는 최상위 레벨에 정확히 하나의 연산자만 허용하므로, 모든 조건을 `$and` 배열로 감싸야 합니다.
    
    # 지침:
    1. 사용자의 질문을 분석하여 필드 정보를 추출하세요.
    
    2. **필터 구조**: 모든 필터는 반드시 다음과 같은 구조를 따라야 합니다:
       {{"$and": [조건1, 조건2, ...]}}
    
    3. **기본 날짜 필터**: 사용자가 기간을 특정하지 않은 모든 경우, 아래 조건들을 **반드시 포함**하세요:
       {{"start_date_int": {{"$lte": {current_date_int}}}}},
       {{"end_date_int": {{"$gte": {current_date_int}}}}}
    
    4. **나이 필터**: 
       - 특정 나이 범위(예: "20대")를 언급하면, 해당 범위의 정책을 찾기 위해:
         {{"min_age": {{"$lte": 최대연령}}}},  // 정책의 최소 나이가 사용자 최대 나이보다 작거나 같음
         {{"max_age": {{"$gte": 최소연령}}}}   // 정책의 최대 나이가 사용자 최소 나이보다 크거나 같음
       - 예: 20대 → min_age <= 29, max_age >= 20
    
    5. **카테고리 필터**:
       - 단순 일치: {{"category_large": "주거"}}
       - 연산자 사용: {{"category_large": {{"$eq": "주거"}}}}
    
    6. **예외 처리**: 사용자가 '과거', '작년', '모든' 등 기간 제한 없이 검색하길 명시적으로 원하면, 날짜 필터를 포함하지 마세요.
    
    7. 추가 설명 없이 오직 JSON 객체만 응답해야 합니다.
    
    # 최종 출력 JSON 예시:
    
    ## 예시 1: "20대 주거 정책 찾아줘" (기본 필터링 적용)
    {{
        "$and": [
            {{"min_age": {{"$lte": 29}}}},
            {{"max_age": {{"$gte": 20}}}},
            {{"category_large": "주거"}},
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    
    ## 예시 2: "현재 신청 가능한 청년 정책" (카테고리 없이)
    {{
        "$and": [
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    
    ## 예시 3: "작년에 했던 모든 정책 알려줘" (기간 제한 없음)
    {{}}
    
    ## 예시 4: "25살이 받을 수 있는 교육 지원"
    {{
        "$and": [
            {{"min_age": {{"$lte": 25}}}},
            {{"max_age": {{"$gte": 25}}}},
            {{"category_large": "교육"}},
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # gpt-4o 또는 gpt-4o-mini 추천
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"LLM 호출 중 오류 발생: {e}")
        return {"error": str(e)}
    
# 최종 지휘자(Orchestrator) 함수
def process_query_hybrid(query: str):
    """하이브리드 방식으로 사용자 질문을 처리하여 최종 ChromaDB 필터를 생성합니다."""
    print(f"입력 질문: \"{query}\"")
    
    if not is_policy_related_query(query):
        print("-> [처리 결과: 규칙 기반 차단]")
        return "정책과 관련된 질문을 해주세요."
    
    print("-> [처리 방식: LLM 호출하여 구조화된 데이터 추출]")
    structured_data = extract_structured_data_with_llm(query)
    print(f"   - LLM 추출 결과 (JSON): {structured_data}")
    return structured_data

In [6]:
user_query = "20대 청년이 받을 수 있는 정책이 궁금해"
ff = process_query_hybrid(user_query)

입력 질문: "20대 청년이 받을 수 있는 정책이 궁금해"
-> [처리 방식: LLM 호출하여 구조화된 데이터 추출]
   - LLM 추출 결과 (JSON): {'$and': [{'min_age': {'$lte': 29}}, {'max_age': {'$gte': 20}}, {'start_date_int': {'$lte': 20250722}}, {'end_date_int': {'$gte': 20250722}}]}


In [18]:
# 필터링 된 리트리버 만드는 함수
# def get_filtered_retriever(filter_condition):
#     return vectorstore.as_retriever(
#         search_type="mmr",
#         search_kwargs={"k":10, "fetch_k":30, "lambda_mult":0.7, "filter": filter_condition}
#     )

def get_filtered_retriever(filter_condition):
    """필터 조건을 받아 MMR 리트리버를 생성하여 반환합니다."""
    print(f"▶ 리트리버 생성 (적용된 필터: {filter_condition})")
    return vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k":10, "fetch_k":30, "lambda_mult":0.7, "filter": filter_condition}
    )

In [14]:
# 테스트
r = get_filtered_retriever(ff)
d = r.invoke(user_query)

for i in d:
    print(i.metadata['start_date_int'])
    print(i.metadata['end_date_int'])
    print('-'*20)

20250101
20250930
--------------------
20250101
20251215
--------------------
20250101
20251210
--------------------
20250101
20251231
--------------------
20250421
20251231
--------------------
20250213
20251231
--------------------
20250203
20251128
--------------------
20250701
20250731
--------------------
20250301
20251130
--------------------
20250101
20251231
--------------------


In [15]:
# chain runnables(Chaining)

# LLM, 프롬프트, 리트리버, 출력 파서 등을 연결하여 하나의 자동화된 흐름을 만드는 것
# LangChain에서의 체이닝은 LCEL에 기반한다.

In [20]:
# 간단한 체인 예제

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableBranch
from langchain_core.documents import Document

# 0. 환경 설정
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 1. 컴포넌트 준비
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 문서 및 벡터 DB 준비(기존에 만들어 둔 벡터DB 사용) => 생략

# 템플릿 준비

template = """주어진 문서 내용만을 바탕으로 질문에 답변해주세요.

[문서 내용]
{context}

[질문]
{question}

[답변]
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# 2. 체인 조립 

# 2-1. guard rail 통과 시 수행할 main chain
main_rag_chain = (
    RunnablePassthrough.assign(
        context=lambda x: get_filtered_retriever(x["eval_result"]).get_relevant_documents(x["question"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

# 2-2. 조건에 따라 분기할 Branch 정의
guardrail_branch = RunnableBranch(
    # 조건: eval_result가 문자열(str)이면 (즉, 차단 메시지이면)
    (lambda x: isinstance(x["eval_result"], str), 
     # 실행할 체인: 그냥 그 차단 메시지를 그대로 반환
     lambda x: x["eval_result"]),
    # 기본값 (else): 위의 조건이 아니면(즉, 필터 dict이면) main_rag_chain을 실행
    main_rag_chain
)

# 2-3. 전체 체인 완성
full_chain = (
    RunnablePassthrough.assign(
        eval_result=lambda x: process_query_hybrid(x["question"])
    )
    | guardrail_branch
)

In [22]:
# --- 3. 체인 실행 ---

# Case 1: Guardrail 통과 (정상 처리)
print("--- Case 1: 정상 질문 실행 ---")
response1 = full_chain.invoke({"question": "20대 주거 정책 알려줘"})
print("\n[최종 답변]:", response1)

print("\n" + "="*50 + "\n")

# Case 2: Guardrail 차단
print("--- Case 2: 관련 없는 질문 실행 ---")
response2 = full_chain.invoke({"question": "오늘 날씨 어때?"})
print("\n[최종 답변]:", response2)

--- Case 1: 정상 질문 실행 ---
입력 질문: "20대 주거 정책 알려줘"
-> [처리 방식: LLM 호출하여 구조화된 데이터 추출]
   - LLM 추출 결과 (JSON): {'$and': [{'min_age': {'$lte': 29}}, {'max_age': {'$gte': 20}}, {'category_large': '주거'}, {'start_date_int': {'$lte': 20250722}}, {'end_date_int': {'$gte': 20250722}}]}
▶ 리트리버 생성 (적용된 필터: {'$and': [{'min_age': {'$lte': 29}}, {'max_age': {'$gte': 20}}, {'category_large': '주거'}, {'start_date_int': {'$lte': 20250722}}, {'end_date_int': {'$gte': 20250722}}]})

[최종 답변]: 주어진 문서에서 20대가 신청할 수 있는 주거 정책은 다음과 같습니다:

1. **강원형 공공주택 공급**
   - 정책 설명: 대학생, 사회초년생, 신혼부부 등 청년층 중심의 공공임대주택 공급으로 주거안정을 통한 지역정착 및 출산·육아 환경 조성
   - 신청 자격: 무주택자 (세부기준은 '공공주택특별법 시행규칙' 별표 5의2 '통합공공임대주택 입주자 자격' 참조)
   - 운영 부서: 강원특별자치도
   - 주관 부서: 국토교통부
   - 신청 기간: 2025년 1월 1일 ~ 2025년 12월 31일

2. **청년 신혼부부 주거자금 대출이자 지원사업**
   - 정책 설명: 청년 신혼부부의 주거비 부담 완화로 안정적 지역 정착을 도모하고 저출생 극복에 기여
   - 신청 자격: 부부 중 1명 이상이 청년(19 ~ 39세)이어야 하며, 혼인신고 7년 이내의 부부
   - 운영 부서: 충주시
   - 신청 기간: 2025년 7월 1일 ~ 2025년 7월 31일

3. **신혼부부 주택전세자금 대출이자 지원사업**
   - 정책 설